In [1]:
import sys
sys.path.append("..")
import argparse
import numpy as np
import dgl
from dgl import DGLGraph
import torch
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from collections import Counter
import pickle
import h5py
import random
import glob2
import seaborn as sns

import train
import models

%load_ext autoreload
%autoreload 2

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
device = train.get_device()

Using backend: pytorch
/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Detected an old version of PyTorch. Suggest using torch>=1.5.0 for the best experience.
  return warnings.warn(message, category=category, stacklevel=1)
/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [3]:
category = "real_data"

epochs = 10
batch_size = 128 
pca_size = 50
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
print(files)
nb_genes =2000

['Muraro', 'Quake_Smart-seq2_Diaphragm', 'worm_neuron_cell', 'Quake_10x_Bladder', 'mouse_bladder_cell', 'Young', '10X_PBMC', 'Quake_10x_Limb_Muscle', 'Adam', 'Quake_Smart-seq2_Trachea', 'Quake_Smart-seq2_Limb_Muscle', 'Quake_10x_Spleen', 'Quake_Smart-seq2_Lung', 'mouse_ES_cell', 'Romanov']


In [4]:
optimal_size = pd.read_pickle(f"../output/pickle_results/{category}/{category}_optimal_size.pkl")

In [5]:
results = pd.DataFrame()
model_name = "GraphConv"
normalize_weights = "log_per_cell"
node_features = "scale"
same_edge_values = False
edge_norm = True
hidden_relu = False
hidden_bn = False
n_layers = 1
hidden_dim = 200
hidden = None  # [300]
nb_genes = 3000
activation = F.relu
for dataset in files:
    print(f">> {dataset}")

    data_mat = h5py.File(f"{path}/real_data/{dataset}.h5", "r")

    Y = np.array(data_mat['Y'])
    X = np.array(data_mat['X'])

    genes_idx, cells_idx = train.filter_data(X, highly_genes=nb_genes)
    X = X[cells_idx][:, genes_idx]
    Y = Y[cells_idx]
    n_clusters = len(np.unique(Y))

    t0 = time.time()
    graph = train.make_graph(
        X,
        Y,
        dense_dim=pca_size,
        node_features=node_features,
        normalize_weights=normalize_weights,
    )

    labels = graph.ndata["label"]
    train_ids = np.where(labels != -1)[0]

    sampler = dgl.dataloading.MultiLayerFullNeighborSampler(n_layers)

    dataloader = dgl.dataloading.NodeDataLoader(
        graph,
        train_ids,
        sampler,
        batch_size=batch_size,
        shuffle=True,
        drop_last=False,
        num_workers=1,
    )
    print(
        f"INPUT: {model_name}  {hidden_dim}, {hidden}, {same_edge_values}, {edge_norm}"
    )
    t1 = time.time()
    for model_name in [
            "GraphConv", "GATConv", "EdgeConv", "SAGEConvMean", "SAGEConvGCN",
            "SAGEConvPool", "GINMax", "GINSum", "GINMean"
    ]:
        for run in range(3):
            t_start = time.time()
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)

            model = models.GAE(in_feats=pca_size,
                               n_hidden=hidden_dim,
                               n_layers=n_layers,
                               activation=activation,
                               dropout=0.1,
                               hidden=hidden,
                               hidden_relu=hidden_relu,
                               hidden_bn=hidden_bn,
                               model=model_name).to(device)
            if run == 0:
                print(f">", model)

            optim = torch.optim.Adam(model.parameters(), lr=1e-5)

            scores = train.train(model,
                                 optim,
                                 epochs,
                                 dataloader,
                                 n_clusters,
                                 plot=False,
                                 cluster=["KMeans", "Leiden"])
            scores["dataset"] = dataset
            scores["run"] = run
            scores["nb_genes"] = nb_genes
            scores["hidden"] = str(hidden)
            scores["hidden_dim"] = str(hidden_dim)
            scores["model_name"] = model_name

            results = results.append(scores, ignore_index=True)

            results.to_pickle(
                f"../output/pickle_results/{category}/{category}_graph_networks.pkl"
            )
            print("Done")

>> Muraro


/opt/conda/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


INPUT: GraphConv  200, None, False, True
> GAE(
  (dropout): Dropout(p=0.1, inplace=False)
  (layer1): GraphConv(in=50, out=200, normalization=both, activation=<function relu at 0x7f2444a760e0>)
  (decoder): InnerProductDecoder()
)


/opt/conda/lib/python3.7/site-packages/umap/__init__.py:9: UserWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn("Tensorflow not installed; ParametricUMAP will be unavailable")


ARI 0.891, 0.5218098759651184
Done


...


ARI 0.6963, 0.40007346868515015
Done



ARI 0.6968, 0.39983057975769043
Done
> GAE(
  (dropout): Dropout(p=0.1, inplace=False)
  (layer1): GINConv(
    (apply_func): Linear(in_features=50, out_features=200, bias=True)
  )
  (decoder): InnerProductDecoder()
)



ARI 0.6191, 0.41621696949005127
Done



ARI 0.6219, 0.42120546102523804
Done



ARI 0.6183, 0.4178329408168793
Done


In [6]:
results.mean()#

ae_end         1.615199e+09
hidden_dim              inf
kmeans_ari     6.157190e-01
kmeans_cal     2.495194e+03
kmeans_nmi     7.127647e-01
kmeans_sil     3.332599e-01
kmeans_time    7.922331e+00
leiden_ari     4.992294e-01
leiden_cal     1.787440e+03
leiden_nmi     6.860104e-01
leiden_sil     2.356598e-01
leiden_time    1.539276e+01
nb_genes       3.000000e+03
run            1.000000e+00
dtype: float64

In [ ]:
results = pd.read_pickle(
                            f"../output/pickle_results/{category}/{category}_graph_networks.pkl"
                        )

In [7]:
results.groupby("model_name").mean()

,ae_end,kmeans_ari,kmeans_cal,kmeans_nmi,kmeans_sil,kmeans_time,leiden_ari,leiden_cal,leiden_nmi,leiden_sil,leiden_time,nb_genes,run
model_name,,,,,,,,,,,,,
EdgeConv,1.615198e+09,0.507131,528.174300,0.652073,0.139573,8.069770,0.512358,400.823357,0.668691,0.113681,15.391204,3000.0,1.0
GATConv,1.615198e+09,0.595631,1107.437482,0.673473,0.322026,8.595260,0.497762,761.847905,0.658229,0.217347,16.005966,3000.0,1.0
GINMax,1.615199e+09,0.434964,455.066843,0.547040,0.146457,8.038113,0.419413,335.303900,0.573476,0.115104,15.553097,3000.0,1.0
GINMean,1.615199e+09,0.679984,4803.871771,0.772853,0.459848,7.779551,0.493691,3517.992237,0.714296,0.322661,15.077321,3000.0,1.0
GINSum,1.615199e+09,0.693660,4026.234874,0.765762,0.430964,7.756650,0.497558,2755.107680,0.711849,0.297787,15.193490,3000.0,1.0
GraphConv,1.615198e+09,0.768760,1907.355806,0.808220,0.429881,7.564166,0.543398,1175.614421,0.730369,0.270722,15.887247,3000.0,1.0
SAGEConvGCN,1.615199e+09,0.648678,4109.413875,0.752298,0.429902,7.873387,0.505176,3039.079853,0.717553,0.307402,15.210460,3000.0,1.0
SAGEConvMean,1.615198e+09,0.674673,4714.268126,0.769424,0.452591,7.691345,0.501024,3492.455959,0.717564,0.323534,14.950689,3000.0,1.0
SAGEConvPool,1.615199e+09,0.537989,804.921451,0.673738,0.188098,7.932739,0.522684,608.731820,0.682067,0.152701,15.265361,3000.0,1.0


In [ ]:
results.groupby("dataset").mean()